<a href="https://colab.research.google.com/github/chongzicbo/Dive-into-Deep-Learning-tf.keras/blob/master/10.6.%20%E6%B1%82%E8%BF%91%E4%B9%89%E8%AF%8D%E5%92%8C%E7%B1%BB%E6%AF%94%E8%AF%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##10.6. 求近义词和类比词
在“word2vec的实现”一节中，我们在小规模数据集上训练了一个word2vec词嵌入模型，并通过词向量的余弦相似度搜索近义词。实际中，在大规模语料上预训练的词向量常常可以应用到下游自然语言处理任务中。本节将演示如何用这些预训练的词向量来求近义词和类比词。我们还将在后面两节中继续应用预训练的词向量。

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import tarfile
print(tf.__version__)

1.15.0


In [0]:
tf.enable_eager_execution()

###10.6.1. 使用预训练的词向量
先下载预训练的中文词向量文件

In [0]:
w2v_glove=keras.utils.get_file('w2v_glove','https://ai.tencent.com/ailab/nlp/data/Tencent_AILab_ChineseEmbedding.tar.gz') #下载词向量

In [5]:
ll /root/.keras/datasets/

total 22970976
-rw-r--r-- 1 root        1805 Oct 19  2018 README.txt
-rw-r--r-- 1 root 16743325366 Oct 19  2018 Tencent_AILab_ChineseEmbedding.txt
-rw-r--r-- 1 root  6778940358 Jan 20 06:24 w2v_glove


In [0]:
tar=tarfile.open('/root/.keras/datasets/w2v_glove','r:gz')
file_names=tar.getnames()
for file_name in file_names:
  tar.extract(file_name,'/root/.keras/datasets/')

该词向量文件较大，我们仅加载100000个单词的词向量

In [0]:
embedding_matrix=np.zeros(shape=[100000,200],dtype='float32')
id_to_word={}
word_to_id={}
f=open("/root/.keras/datasets/Tencent_AILab_ChineseEmbedding.txt")
i=0
for line in f:
  if i==100000:
    break
  values=line.split()
  if len(values)<201:
    continue
  id_to_word[i]=values[0]
  word_to_id[values[0]]=i
  #每行的第一个元素是词，后面才是词向量，因此将values[0]与values[1]分开存放
  coefs=np.asarray(values[1:],dtype='float32')
  embedding_matrix[i]=coefs
  i+=1


f.close()


打印一个词向量看看，词向量维度为200

In [62]:
embedding_matrix[-2],list(vocablary.values())[-2]

(array([ 0.468819, -0.375571,  0.134898,  0.275068, -0.046392,  0.397821,
        -0.131702,  0.122381, -0.402547, -0.857406, -0.347803,  0.193127,
        -0.381756, -0.322535,  0.320485, -0.141673, -0.415495, -0.111749,
         0.004892,  0.244213,  0.109055,  0.054361, -0.077072,  0.432907,
         0.077875, -0.012601, -0.22354 ,  0.179577,  0.788958,  0.503813,
        -0.292133, -0.103239,  0.351905, -0.091495,  0.195276,  0.311701,
         0.196822,  0.003728, -0.268067,  0.036879,  0.160408,  0.129077,
         0.722161,  0.296376, -0.039628, -0.612891, -0.318631, -0.653084,
        -0.044348, -0.117665, -0.29183 , -0.515927,  0.219945,  0.097836,
         0.783008, -0.094529, -0.207038,  0.290682,  0.289661,  0.161448,
         0.21637 ,  0.030424,  0.290693,  0.111558, -0.427859, -0.315404,
         0.145403,  0.252281,  0.027503, -0.119779, -0.493411, -0.153594,
         0.453617,  0.291261,  0.034325, -0.078553,  0.022672, -0.220923,
         0.091114, -0.038166,  0.12960

###10.6.2. 应用预训练词向量
下面我们以GloVe模型为例，展示预训练词向量的应用。

####10.6.2.1. 求近义词
这里重新实现“word2vec的实现”一节中介绍过的使用余弦相似度来搜索近义词的算法。为了在求类比词时重用其中的求 k 近邻（ k -nearest neighbors）的逻辑，我们将这部分逻辑单独封装在knn函数中。

In [0]:
def knn(W,x,k):
  #添加的1e-9是为了数值稳定性
  cos=np.dot(W,x.reshape((-1,)))/(np.sqrt(np.sum(W*W,axis=1)+1e-9)*np.sqrt(np.sum(x*x)))
  return np.argsort(-cos)[0:k]#降序排序

In [0]:
x=embedding_matrix[1]
topk=knn(embedding_matrix,x,k=3)

然后，我们通过预训练词向量来搜索近义词。

In [0]:
def get_similar_tokens(query_token,k,embedding_matrix):
  id=word_to_id[query_token]
  x=embedding_matrix[id]
  topk=knn(embedding_matrix,x,k+1)
  for i in topk[1:]:
    print(id_to_word[i])

In [72]:
get_similar_tokens('办理',3,embedding_matrix)

手续
申请办理
办理手续


####10.6.2.2. 求类比词
除了求近义词以外，我们还可以使用预训练词向量求词与词之间的类比关系。例如，“man”（男人）: “woman”（女人）:: “son”（儿子） : “daughter”（女儿）是一个类比例子：“man”之于“woman”相当于“son”之于“daughter”。求类比词问题可以定义为：对于类比关系中的4个词$a : b :: c : d$，给定前3个词 $a 、 b$ 和 $c$ ，求 $d$ 。设词 $w$ 的词向量为$ vec(w)$ 。求类比词的思路是，搜索与$\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$的结果向量最相似的词向量。

In [0]:
def get_analogy(token_a,token_b,token_c,embedding_matrix):
  vec_a=embedding_matrix[word_to_id[token_a]]
  vec_b=embedding_matrix[word_to_id[token_b]]
  vec_c=embedding_matrix[word_to_id[token_c]]
  x=vec_c+vec_b-vec_a
  topk=knn(embedding_matrix,x,2)
  for i in topk[0:]:
    print(id_to_word[i])
  # return id_to_word[topk[0]]

验证一下“男-女”类比。

In [89]:
get_analogy('男人','女人','儿子',embedding_matrix)

儿子
女儿


“首都-国家”类比

In [94]:
get_analogy('中国','北京','日本',embedding_matrix)

北京
东京


###10.6.3. 小结
* 在大规模语料上预训练的词向量常常可以应用于下游自然语言处理任务中。
* 可以应用预训练的词向量求近义词和类比词。